# Recuperación de pacientes de XiO
-------------------
## Descripción
Mediante este cuaderno datos de *pacientes archivados en XiO se convierten a formato DICOM* para su importación en ARIA.

Los datos archivados en XiO están almacenados en un servidor del Hospital al que se accede mediante `ftp`

La conversión utiliza los programas `plastimatch`y `Velocity`

`plastimatch` recupera de los datos de XiO las imágenes de CT, las estructuras y la dosis.

`Velocity` añade el plan contenedor para poder importar la dosis en ARIA.

## Procedimiento

* Recuperar los datos archivados del servidor del Hospital
* Desarchivar los datos de XiO
* Correr los scripts de recuperación de las imágenes CT, las estructuras y la dosis
* Fijar un marco de referencia común para las imágenes CT, las estructuras y la dosis
* Crear en Velocity un plan contenedor para la dosis
* Importar en ARIA
  
### Recuperación de los datos archivados

Esta fase requiere del uso de un cliente `ftp`.

Se recomienda usar `FileZilla`

Los datos de conexión son:
* Servidor: 10.137.2.113
* Nombre de usuario: radioterapia
* Contraseña: radioterapia

Pulsar sobre el botón `Conexión rápida`

En Sitio remoto navegar en la ruta `Archivo/No Categorizado` para seleccionar la carpeta del paciente que se quiere recuperar

Seleccionar el archivo`.tgz`. Si hay varios elegir el más reciente

Arrastrar el archivo sobre la carpeta `Radiofisica/XiOAriaDrive` del Sitio Local

![FileZillaMainScreen](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/FileZillaMainScreen.PNG) )
### Desarchivar los datos de XiO

Definir el directorio de trabajo base

In [2]:
# Referenciar
baseworkdir = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO'

from pathlib import Path
if not Path(baseworkdir).exists():
    !mkdir -p $baseworkdir

Definir el nombre del archivo con los datos archivados en XiO

In [3]:
ArchivedPatient = '584953_SALDA_A_GARCES_SANTIAGO_Mar_14_2011.tgz'

Desarchivar

In [4]:
%%capture
!tar -xvzf $ArchivedPatient -C $baseworkdir

Mostrar la estructura de carpetas de los datos desarchivados

In [5]:
!tree -d $baseworkdir/patient

/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient
└── 584953
    ├── anatomy
    │   └── studyset
    │       └── CT1
    │           ├── DCMData
    │           └── surfaces
    ├── dvh
    └── plan
        ├── 1FASE1B
        └── PLANSETUP

10 directories


Recuperar datos relevantes.

Número de historia y nombre y apellidos del paciente

In [12]:
# Número de historia
PatientID = ArchivedPatient.split('_')[0]

# Nombre y apellidos
demographicfile = !find $baseworkdir/patient/$PatientID -name demographic -print
with open(demographicfile[0], 'r', encoding='latin') as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]

print(f'Historia: {PatientID}\nApellidos y nombre: {ApellidosNombre}')

Historia: 584953
Apellidos y nombre: SALDAÑA GARCES,SANTIAGO


Definir el estudio CT y el plan que se quiere recuperar

In [7]:
CT = 'CT1'
Plan ='1FASE1B'

### Scripts de recuperación

Importación de módulos

In [8]:
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast
# - Rutas y comprobaciones de existencia de ficheros
from os import path, getcwd
from glob import glob
from sys import path as syspath

Crear la carpeta `scripts` en el directorio de trabajo base y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [9]:
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py

# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

--2023-08-09 14:22:27--  https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
Resolving proxy.salud.madrid.org (proxy.salud.madrid.org)... 172.22.200.10
Connecting to proxy.salud.madrid.org (proxy.salud.madrid.org)|172.22.200.10|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 7741 (7,6K) [text/plain]
Saving to: ‘/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/scripts/xioss’

xioss               100%[===================>]   7,56K  --.-KB/s    in 0,005s  

2023-08-09 14:22:27 (1,63 MB/s) - ‘/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/scripts/xioss’ saved [7741/7741]

--2023-08-09 14:22:27--  https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
Resolving proxy.salud.madrid.org (proxy.salud.madrid.org)... 172.22.200.10
Connecting to proxy.salud.madrid.org (proxy.salud.madrid.org)|172.22.200.10|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 612 [text/plain]
Saving to: ‘/home/rad

Recuperación de las estructuras

In [14]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
    # Copiar el script para la creación del índice de cortes de CT
    !cp $baseworkdir/scripts/genindex $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    
    # Adjudicar permisos de ejecución y crear el índice
    !chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/genindex
    cwd = getcwd()
    %cd -q $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    print('Generar el índice de imáagenes de CT...')
    !./genindex
    %cd -q $cwd

# Crear la carpeta para los archivos máscara con las estructuras
!mkdir -p $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/ss

# Copiar el script para la creación de las estructuras
!cp $baseworkdir/scripts/xioss $baseworkdir/patient/$PatientID/anatomy/studyset/$CT

# Correr el script de recuperación de las estructuras
!chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss
print('Recuperar estructuras...')
!$baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss --input $baseworkdir/patient/$PatientID --plan $Plan --studyset $CT > xioss.log

Recuperar estructuras...
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/EXTERNO.mha' and 'ss/EXTERNO.mha' are the same file
mv: 'ss/pulmon dcho.mha' and 'ss/pulmon dcho.mha' are the same file
mv: 'ss/pulmon izqdo.mha' and 'ss/pulmon izqdo.mha' are the same file
mv: 'ss/medula.mha' and 'ss/medula.mha' are the same file
mv: 'ss/vertebra.mha' and 'ss/vertebra.mha' are the same file
mv: 'ss/corazon.mha' and 'ss/corazon.mha' are the same file
mv: 'ss/esofago.mha' and 'ss/esofago.mha' are the same file
mv: 'ss/2+4+10+11 R.mha' and 'ss/2+4+10+11 R.mha' are the same file
mv: 'ss/PTV 50.4.mha' and 'ss/PTV 50.4.mha' are the same file
mv: 'ss/3+5.mha' and 'ss/3+5.mha' are the same file
Use of uninitialized value in string eq at /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1/xioss line 152.
Consistency lack in structure match between XiO contour file and plastimatch prefix output at /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/Xi

Conversion del CT, las estructuas y la dosis desde formato XIO a formato DICOM 

In [15]:
# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                  'input-prefix' : baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                  'input-dose-xio' : baseworkdir + '/patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : baseworkdir + '/../' + PatientID + '/plastimatch'}

# Correr el script de conversión de los datos XiO en estudio DICOM
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )


Running 'plastimatch convert' with the specified arguments:
  --input /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1/DCMData
  --input-prefix /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1/ss
  --input-dose-xio /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/plan/1FASE1B/dose.1
  --output-dicom /home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/../584953/plastimatch
... Done.


### Corrección de los marcos de referencia

In [16]:
# Referencia al directorio de trabajo actual
cwd = getcwd()
# Cambiar al directorio patient
%cd -q $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd -q $baseworkdir

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages

# Establecer el mismo marco de referencia en todo el estudio
print('Establecer el mismo marco de referencia para las imágenes CT, las estructuras y la dosis...')
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT, dcmdir=baseworkdir + '/../' + PatientID + '/plastimatch')

# Volver al directorio inicial
%cd -q $cwd

Establecer el mismo marco de referencia para las imágenes CT, las estructuras y la dosis...


### Crear plan contenedor (Velocity)

### Importar en ARIA